<a href="https://colab.research.google.com/github/raj-vijay/da/blob/master/05_Boolean_Satisfiability_and_Planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**The wolf, the sheep, and the cabbage**

<p align = 'justify'>On the shore of a river, there is a wolf, a sheep, and a
cabbage. One can take a maximum of one of these with them on a boat. Obviously, the objective is to ferry all across, and hence would need multiple journeys back and forth. 

The problem is, if you leave the wolf and the sheep unattended, the wolf will eat the sheep. Also, if you leave the sheep and the cabbage alone on one shore, the sheep will have eaten the cabbage when you return.</p>

The plan is to answer how to ferry all three over the river, never leaving wolf and sheep or sheep and cabbage unattended.

![alt text](https://i.ytimg.com/vi/pU-5OBl_ucM/sddefault.jpg)

<p align = 'justify'>Write a Python program that creates a CP-SAT model and adds all Boolean variables necessary for representing the predicates required for this problem to this model.</p>

In [ ]:
!pip install ortools

     |████████████████████████████████| 13.7MB 302kB/s 
     |████████████████████████████████| 1.3MB 42.8MB/s 
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


In [ ]:
from ortools.sat.python import cp_model

In [ ]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.variables_ = variables
        self.solutions_ = 0

    def OnSolutionCallback(self):
        self.solutions_ = self.solutions_ + 1
        print("solution", self.solutions_ )
        i=0
        for vars_in_timestep in self.variables_:
            i=i+1
            # print(" - Timestep: ", i)
            for op in vars_in_timestep:
                if self.Value(vars_in_timestep[op]):
                    print("   ", op)
        print()

<p align = 'justify'>Create the additional Boolean variables required to model the operators of this problem and add them to the CP-SAT model.</p>

In [ ]:
model = cp_model.CpModel()

In [ ]:
maxT = 8

In [ ]:
WolfOnThisSide = []   
SheepOnThisSide = []
CabbageOnThisSide = []
FerrymanOnThisSide = []

In [ ]:
WolfOnOppositeSide = []
SheepOnOppositeSide = []
CabbageOnOppositeSide = []
FerrymanOnOppositeSide = []

In [ ]:
for t in range(maxT):
  WolfOnThisSide.append(model.NewBoolVar("WolfOnThisSide"+str(t)))
  SheepOnThisSide.append(model.NewBoolVar("SheepOnThisSide"+str(t)))
  CabbageOnThisSide.append(model.NewBoolVar("CabbageOnThisSide"+str(t)))
  FerrymanOnThisSide.append(model.NewBoolVar("FerrymanOnThisSide"+str(t)))
  
  WolfOnOppositeSide.append(model.NewBoolVar("WolfOnOppositeSide"+str(t)))
  SheepOnOppositeSide.append(model.NewBoolVar("SheepOnOppositeSide"+str(t)))
  CabbageOnOppositeSide.append(model.NewBoolVar("CabbageOnOppositeSide"+str(t)))
  FerrymanOnOppositeSide.append(model.NewBoolVar("FerrymanOnOppositeSide"+str(t)))

<p align = 'justify'>Formulate the initial and goal state in terms of the variables created in task 1 and add them to the CP-SAT model as constraints.</p>

In [ ]:
# Initial state
model.AddBoolAnd(
        [FerrymanOnThisSide[0], 
         WolfOnThisSide[0], 
         SheepOnThisSide[0], 
         CabbageOnThisSide[0] ] )

In [ ]:
model.AddBoolAnd( 
  [FerrymanOnOppositeSide[0].Not(), 
   WolfOnOppositeSide[0].Not(), 
   SheepOnOppositeSide[0].Not(), 
   CabbageOnOppositeSide[0].Not() ] )

In [ ]:
# Goal state
model.AddBoolAnd(
    [WolfOnOppositeSide[maxT-1],
     SheepOnOppositeSide[maxT-1],
     CabbageOnOppositeSide[maxT-1]])

<p align = 'justify'>Create the additional Boolean variables required to model the operators of this problem and add them to the CP-SAT model.</p>

In [ ]:
# Operator encodings
moveWolfAccross = []
moveSheepAccross = []
moveCabbageAccross = []
moveWolfBack = []
moveSheepBack = []
moveCabbageBack = []
moveAccross = []
moveBack = []

<p align = 'justify'>Formulate the pre- and post-conditions for all operators defined and add the resulting operator encodings to the CP-SAT model.</p>

In [ ]:
for t in range(maxT-1):
  moveWolfAccross.append(model.NewBoolVar("MoveWolfAccross"+str(t)))
  moveSheepAccross.append(model.NewBoolVar("MoveSheepAccross"+str(t)))
  moveCabbageAccross.append(model.NewBoolVar("MoveCabbageAccross"+str(t)))
  moveWolfBack.append(model.NewBoolVar("MoveWolfBack"+str(t)))
  moveSheepBack.append(model.NewBoolVar("MoveSheepBack"+str(t)))
  moveCabbageBack.append(model.NewBoolVar("MoveCabbageBack"+str(t)))
  moveAccross.append(model.NewBoolVar("MoveAccross"+str(t)))
  moveBack.append(model.NewBoolVar("MoveBack"+str(t)))
  
  model.AddBoolAnd([WolfOnThisSide[t],FerrymanOnThisSide[t],
                    WolfOnOppositeSide[t+1], FerrymanOnOppositeSide[t+1], 
                    WolfOnThisSide[t+1].Not(), FerrymanOnThisSide[t+1].Not()
                    ]).OnlyEnforceIf(moveWolfAccross[t])
  model.AddBoolAnd([WolfOnOppositeSide[t], FerrymanOnOppositeSide[t],
                    WolfOnThisSide[t+1], FerrymanOnThisSide[t+1], 
                    WolfOnOppositeSide[t+1].Not(), FerrymanOnOppositeSide[t+1].Not()
                    ]).OnlyEnforceIf(moveWolfBack[t])
  model.AddBoolAnd([SheepOnThisSide[t], FerrymanOnThisSide[t],
                    SheepOnOppositeSide[t+1], FerrymanOnOppositeSide[t+1], SheepOnThisSide[t+1].Not(), FerrymanOnThisSide[t+1].Not()
                    ]).OnlyEnforceIf(moveSheepAccross[t])
  model.AddBoolAnd([SheepOnOppositeSide[t], FerrymanOnOppositeSide[t],
                    SheepOnThisSide[t+1], FerrymanOnThisSide[t+1], SheepOnOppositeSide[t+1].Not(), FerrymanOnOppositeSide[t+1].Not()
                    ]).OnlyEnforceIf(moveSheepBack[t])

  model.AddBoolAnd([CabbageOnThisSide[t], FerrymanOnThisSide[t],
                    CabbageOnOppositeSide[t+1], FerrymanOnOppositeSide[t+1], CabbageOnThisSide[t+1].Not(), FerrymanOnThisSide[t+1].Not()
                    ]).OnlyEnforceIf(moveCabbageAccross[t])
  
  model.AddBoolAnd([CabbageOnOppositeSide[t], FerrymanOnOppositeSide[t],
                    CabbageOnThisSide[t+1], FerrymanOnThisSide[t+1], CabbageOnOppositeSide[t+1].Not(), FerrymanOnOppositeSide[t+1].Not()
                    ]).OnlyEnforceIf(moveCabbageBack[t])
                    
  model.AddBoolAnd([FerrymanOnThisSide[t],
                    FerrymanOnOppositeSide[t+1], FerrymanOnThisSide[t+1].Not()
                    ]).OnlyEnforceIf(moveAccross[t])
  model.AddBoolAnd([FerrymanOnOppositeSide[t],
                    FerrymanOnThisSide[t+1], FerrymanOnOppositeSide[t+1].Not()
                    ]).OnlyEnforceIf(moveBack[t])  

Formulate the frame axioms for the predicates defined earlier and add them as constraints to the CP-SAT model.

In [ ]:
# Frame axioms (no state is switched on without an action)
for t in range(maxT-1):
  model.AddBoolOr([WolfOnThisSide[t+1].Not(), WolfOnThisSide[t], moveWolfBack[t]])
  model.AddBoolOr([WolfOnOppositeSide[t+1].Not(), WolfOnOppositeSide[t], moveWolfAccross[t]])
  model.AddBoolOr([SheepOnThisSide[t+1].Not(), SheepOnThisSide[t], moveSheepBack[t]])
  model.AddBoolOr([SheepOnOppositeSide[t+1].Not(), SheepOnOppositeSide[t], moveSheepAccross[t]])
  model.AddBoolOr([CabbageOnThisSide[t+1].Not(), CabbageOnThisSide[t], moveCabbageBack[t]])
  model.AddBoolOr([CabbageOnOppositeSide[t+1].Not(), CabbageOnOppositeSide[t], moveCabbageAccross[t]])
  model.AddBoolOr([FerrymanOnThisSide[t+1].Not(), 
                   FerrymanOnThisSide[t], 
                   moveWolfBack[t], 
                   moveSheepBack[t], 
                   moveCabbageBack[t], 
                   moveBack[t]])
  model.AddBoolOr([FerrymanOnOppositeSide[t+1].Not(), 
                   FerrymanOnOppositeSide[t], 
                   moveWolfAccross[t], 
                   moveSheepAccross[t], 
                   moveCabbageAccross[t], 
                   moveAccross[t]]) 

Formulate the complete exclusion axioms for the operators defined in task 3 and add them as constraints to the CP-SAT model.

In [ ]:
# Complete exclusion axiom (only one action at a time)
for t in range(maxT-1):
  model.AddBoolOr([moveWolfAccross[t].Not(), moveSheepAccross[t].Not()])
  model.AddBoolOr([moveWolfAccross[t].Not(), moveCabbageAccross[t].Not()])
  model.AddBoolOr([moveWolfAccross[t].Not(), moveWolfBack[t].Not()])
  model.AddBoolOr([moveWolfAccross[t].Not(), moveSheepBack[t].Not()])
  model.AddBoolOr([moveWolfAccross[t].Not(), moveCabbageBack[t].Not()])
  model.AddBoolOr([moveWolfAccross[t].Not(), moveAccross[t].Not()])
  model.AddBoolOr([moveWolfAccross[t].Not(), moveBack[t].Not()])
  model.AddBoolOr([moveSheepAccross[t].Not(), moveCabbageAccross[t].Not()])
  model.AddBoolOr([moveSheepAccross[t].Not(), moveWolfBack[t].Not()])
  model.AddBoolOr([moveSheepAccross[t].Not(), moveSheepBack[t].Not()])
  model.AddBoolOr([moveSheepAccross[t].Not(), moveCabbageBack[t].Not()])
  model.AddBoolOr([moveSheepAccross[t].Not(), moveAccross[t].Not()])
  model.AddBoolOr([moveSheepAccross[t].Not(), moveBack[t].Not()])
  model.AddBoolOr([moveCabbageAccross[t].Not(), moveWolfBack[t].Not()])
  model.AddBoolOr([moveCabbageAccross[t].Not(), moveSheepBack[t].Not()])
  model.AddBoolOr([moveCabbageAccross[t].Not(), moveCabbageBack[t].Not()])
  model.AddBoolOr([moveCabbageAccross[t].Not(), moveAccross[t].Not()])
  model.AddBoolOr([moveCabbageAccross[t].Not(), moveBack[t].Not()])
  model.AddBoolOr([moveWolfBack[t].Not(), moveSheepBack[t].Not()])
  model.AddBoolOr([moveWolfBack[t].Not(), moveCabbageBack[t].Not()])
  model.AddBoolOr([moveWolfBack[t].Not(), moveAccross[t].Not()])
  model.AddBoolOr([moveWolfBack[t].Not(), moveBack[t].Not()])
  model.AddBoolOr([moveSheepBack[t].Not(), moveCabbageBack[t].Not()])
  model.AddBoolOr([moveSheepBack[t].Not(), moveAccross[t].Not()])
  model.AddBoolOr([moveSheepBack[t].Not(), moveBack[t].Not()])
  model.AddBoolOr([moveCabbageBack[t].Not(), moveAccross[t].Not()])
  model.AddBoolOr([moveCabbageBack[t].Not(), moveBack[t].Not()])
  model.AddBoolOr([moveAccross[t].Not(), moveBack[t].Not()])

Formulate the additional constraints that neither the wolf and the sheep nor the sheep and the cabbage can ever be left alone and add them as constraints to the CP-SAT model.

In [ ]:
# Additional constraints (wolf eats sheep, sheep eats cabbage)
for t in range(maxT):
  model.AddBoolOr([WolfOnThisSide[t].Not(),
                   SheepOnThisSide[t].Not()]).OnlyEnforceIf(FerrymanOnThisSide[t].Not())
  model.AddBoolOr([WolfOnOppositeSide[t].Not(),
                   SheepOnOppositeSide[t].Not()]).OnlyEnforceIf(FerrymanOnOppositeSide[t].Not())
  model.AddBoolOr([SheepOnThisSide[t].Not(), 
                   CabbageOnThisSide[t].Not()]).OnlyEnforceIf(FerrymanOnThisSide[t].Not())
  model.AddBoolOr([SheepOnOppositeSide[t].Not(),
                   CabbageOnOppositeSide[t].Not()]).OnlyEnforceIf(FerrymanOnOppositeSide[t].Not())

In [ ]:
variables = []

In [ ]:
for t in range(maxT-1):
  variables.append(
      {
       # "Ferryman on this side": FerrymanOnThisSide[t],
       # "Ferryman on opposite side": FerrymanOnOppositeSide[t],
       # "Wolf on this side": WolfOnThisSide[t],
       # "Wolf on opposite side": WolfOnOppositeSide[t],
       # "Sheep on this side": SheepOnThisSide[t],
       # "Sheep on opposite side": SheepOnOppositeSide[t],
       # "Cabbagge on this side": CabbageOnThisSide[t],
       # "Cabbage on opposite side": CabbageOnOppositeSide[t],
       "move accross":moveAccross[t],
       "move back":moveBack[t],
       "move wolf accross":moveWolfAccross[t],
       "move wolf back":moveWolfBack[t],
       "move sheep accross":moveSheepAccross[t],
       "move sheep back": moveSheepBack[t],
       "move cabbage accross": moveCabbageAccross[t],
       "move cabbage back": moveCabbageBack[t]              
       })

Implement a CpSolverSolutionCallback that prints out the sequence of operations indicated by the variables identified earlier and run the solver for the model and print out the results.

In [ ]:
solver = cp_model.CpSolver()    
solver.SearchForAllSolutions(model, SolutionPrinter(variables))

solution 1
    move sheep accross
    move back
    move wolf accross
    move sheep back
    move cabbage accross
    move back
    move sheep accross

solution 2
    move sheep accross
    move back
    move cabbage accross
    move sheep back
    move wolf accross
    move back
    move sheep accross



4

In [ ]:
for t in range(maxT-1):
  if solver.Value(moveWolfAccross[t]): print(t, ">> move wolf accross")
  if solver.Value(moveWolfBack[t]): print(t, ">> move wolf back")
  if solver.Value(moveSheepAccross[t]): print(t, ">> move sheep accross")
  if solver.Value(moveSheepBack[t]): print(t, ">> move sheep back")
  if solver.Value(moveCabbageAccross[t]): print(t, ">> move cabbage accross")
  if solver.Value(moveCabbageBack[t]): print(t, ">> move cabbage back")
  if solver.Value(moveAccross[t]): print(t, ">> move accross")
  if solver.Value(moveBack[t]): print(t, ">> move back")

0 >> move sheep accross
1 >> move back
2 >> move cabbage accross
3 >> move sheep back
4 >> move wolf accross
5 >> move back
6 >> move sheep accross
